In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib

In [ ]:
def extract_features(map_file, atlas_file, atlas_labels_file, measure_method='mean'):
    """
    Extract feature values for each area in the brain atlas.
    """
    # Load the map
    map_img = nib.load(map_file)
    map_data = map_img.get_fdata()
    
    # Load the atlas
    atlas_img = nib.load(atlas_file)
    atlas_data = atlas_img.get_fdata()
    
    # Load the atlas labels
    atlas_labels = pd.read_csv(atlas_labels_file)
    
    # Create a dictionary to store feature values
    feature_values = {}

    # Select the appropriate measurement function
    if measure_method == 'mean':
        measure_func = np.mean
    elif measure_method == 'sum':
        measure_func = np.sum
    elif measure_method == 'median':
        measure_func = np.median
    elif measure_method == 'max':
        measure_func = np.max
    elif measure_method == 'min':
        measure_func = np.min
    elif measure_method == 'std':
        measure_func = np.std
    else:
        raise ValueError(f"Unknown measure method: {measure_method}")

    # For each area in the atlas, extract the corresponding feature values
    for _, row in atlas_labels.iterrows():
        label = row['Label']
        name = row['Name']
        
        # Create binary mask for current rarea
        area_mask = (atlas_data == label)
        
        # Extract feature values for the area
        area_feature_values = map_data[area_mask]
        
        # Calculate summary value for the area (if the area exists in the mask)
        if area_feature_values.size > 0:
            feature_values[name] = measure_func(area_feature_values)
        else:
            feature_values[name] = np.nan
    
    # Convert to DataFrame
    features_df = pd.DataFrame([feature_values])
    
    return features_df

In [ ]:
# Paths to the files
map_file = "GM.nii"
atlas_file = "Hammers_60.nii"
atlas_labels_file = "Hammers_60.csv"
output_file = "sMRI_Features.csv"

# Single subject processing
features_df = extract_features(map_file, atlas_file, atlas_labels_file, measure_method='sum')
features_df.to_csv(output_file, index=False)
print(f"Successfully saved results to {output_file}")